# Gender dynamics

## Tweeter data prep

### Load the tweets

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import logging
from dateutil.parser import parse as date_parse
from utils import load_tweet_df, tweet_type
import matplotlib.pyplot as plt


logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Set float format so doesn't display scientific notation
pd.options.display.float_format = '{:20,.2f}'.format

def tweet_transform(tweet):
    return {
        'tweet_id': tweet['id_str'], 
        'tweet_created_at': date_parse(tweet['created_at']),
        'user_id': tweet['user']['id_str'],
        'screen_name': tweet['user']['screen_name'],
        'tweet_type': tweet_type(tweet)
    }

tweet_df = load_tweet_df(tweet_transform, ['tweet_id', 'user_id', 'screen_name', 'tweet_created_at', 'tweet_type'], dedupe_columns=['tweet_id'])
tweet_df.count()

INFO:root:Loading from tweets/41feff28312c433ab004cd822212f4c2_001.json.gz
DEBUG:root:Loaded 50000
DEBUG:root:Loaded 100000
DEBUG:root:Loaded 150000
DEBUG:root:Loaded 200000
DEBUG:root:Loaded 250000
DEBUG:root:Loaded 300000
INFO:root:Loading from tweets/642bf140607547cb9d4c6b1fc49772aa_001.json.gz
DEBUG:root:Loaded 350000
DEBUG:root:Loaded 400000
DEBUG:root:Loaded 450000
DEBUG:root:Loaded 500000
DEBUG:root:Loaded 550000
INFO:root:Loading from tweets/9f7ed17c16a1494c8690b4053609539d_001.json.gz
DEBUG:root:Loaded 600000
DEBUG:root:Loaded 650000
DEBUG:root:Loaded 700000
DEBUG:root:Loaded 750000
DEBUG:root:Loaded 800000


tweet_id            817136
user_id             817136
screen_name         817136
tweet_created_at    817136
tweet_type          817136
dtype: int64

In [2]:
tweet_df.head()

,tweet_id,user_id,screen_name,tweet_created_at,tweet_type
0,875028934177542144,15731368,HowardKurtz,2017-06-14 16:35:29+00:00,original
1,875012559228874752,15731368,HowardKurtz,2017-06-14 15:30:25+00:00,retweet
2,875012374343929862,15731368,HowardKurtz,2017-06-14 15:29:41+00:00,original
3,875010759759101952,15731368,HowardKurtz,2017-06-14 15:23:16+00:00,retweet
4,875010163496939520,15731368,HowardKurtz,2017-06-14 15:20:54+00:00,retweet


## Prepare the tweeter data
This comes from the following sources:
1. User lookup: These are lists of users exported from SFM. These are the final set of beltway journalists. Accounts that were suspended or deleted have been removed from this list. Also, this list will include users that did not tweet (i.e., have no tweets in dataset).
2. Tweets in the dataset: Used to generate tweet counts per tweeter. However, since some beltway journalists may not have tweeted, this may be a subset of the user lookup. Also, it may include the tweets of some users that were later excluded because their accounts were suspended or deleted or determined to not be beltway journalists.
3. User info lookup: Information on users that was manually coded in the beltway journalist spreadsheet or looked up from Twitter's API. This includes some accounts that were excluded from data collection for various reasons such as working for a foreign news organization or no longer working as a beltway journalist. Thus, these are a superset of the user lookup.

Thus, the tweeter data should include tweet and user info data only from users in the user lookup.

### Load user lookup

In [3]:
user_lookup_filepaths = ('lookups/senate_press_lookup.csv',
                         'lookups/periodical_press_lookup.csv',
                         'lookups/radio_and_television_lookup.csv')
user_lookup_df = pd.concat((pd.read_csv(user_lookup_filepath, usecols=['Uid', 'Token'], dtype={'Uid': str}) for user_lookup_filepath in user_lookup_filepaths))
user_lookup_df.set_index('Uid', inplace=True)
user_lookup_df.rename(columns={'Token': 'screen_name'}, inplace=True)
user_lookup_df.index.names = ['user_id']
# Some users may be in multiple lists, so need to drop duplicates
user_lookup_df = user_lookup_df[~user_lookup_df.index.duplicated()]

user_lookup_df.count()

screen_name    2487
dtype: int64

In [4]:
user_lookup_df.head()

,screen_name
user_id,
23455653,abettel
33919343,AshleyRParker
18580432,b_fung
399225358,b_muzz
18834692,becca_milfeld


### Tweets in dataset per tweeter

In [5]:
user_tweet_count_df = tweet_df[['user_id', 'tweet_type']].groupby(['user_id', 'tweet_type']).size().unstack()
user_tweet_count_df.fillna(0, inplace=True)
user_tweet_count_df['tweets_in_dataset'] = user_tweet_count_df.original + user_tweet_count_df.quote + user_tweet_count_df.reply + user_tweet_count_df.retweet
user_tweet_count_df.count()

tweet_type
original             2292
quote                2292
reply                2292
retweet              2292
tweets_in_dataset    2292
dtype: int64

In [6]:
user_tweet_count_df.head()

tweet_type,original,quote,reply,retweet,tweets_in_dataset
user_id,,,,,
1001991865,13.00,3.00,1.00,31.00,48.00
1002229862,48.00,20.00,3.00,118.00,189.00
100270054,1.00,0.00,0.00,0.00,1.00
100802089,4.00,7.00,12.00,17.00,40.00
100860790,102.00,26.00,4.00,166.00,298.00


### Load user info

In [7]:
user_info_df = pd.read_csv('source_data/user_info_lookup.csv', names=['user_id', 'name', 'organization', 'position',
                                            'gender', 'followers_count', 'following_count', 'tweet_count',
                                            'user_created_at', 'verified', 'protected'],
                          dtype={'user_id': str}).set_index(['user_id'])
user_info_df.count()

name               2506
organization       2477
position           2503
gender             2505
followers_count    2506
following_count    2506
tweet_count        2506
user_created_at    2506
verified           2506
protected          2506
dtype: int64

In [8]:
user_info_df.head()

,name,organization,position,gender,followers_count,following_count,tweet_count,user_created_at,verified,protected
user_id,,,,,,,,,,
20711445,"Glinski, Nina",NaN,Freelance Reporter,F,963,507,909,Thu Feb 12 20:00:53 +0000 2009,False,False
258917371,"Enders, David",NaN,Journalist,M,1444,484,6296,Mon Feb 28 19:52:03 +0000 2011,True,False
297046834,"Barakat, Matthew",Associated Press,Northern Virginia Correspondent,M,759,352,631,Wed May 11 20:55:24 +0000 2011,True,False
455585786,"Atkins, Kimberly",Boston Herald,Chief Washington Reporter/Columnist,F,2944,2691,6277,Thu Jan 05 08:26:46 +0000 2012,True,False
42584840,"Vlahou, Toula",CQ Roll Call,Editor & Podcast Producer,F,2703,201,6366,Tue May 26 07:41:38 +0000 2009,False,False


In [9]:
user_summary_df = user_lookup_df.join((user_info_df, user_tweet_count_df), how='left')
# Fill Nans
user_summary_df['organization'].fillna('', inplace=True)
user_summary_df['original'].fillna(0, inplace=True)
user_summary_df['quote'].fillna(0, inplace=True)
user_summary_df['reply'].fillna(0, inplace=True)
user_summary_df['retweet'].fillna(0, inplace=True)
user_summary_df['tweets_in_dataset'].fillna(0, inplace=True)
user_summary_df.count()

screen_name          2487
name                 2487
organization         2487
position             2484
gender               2486
followers_count      2487
following_count      2487
tweet_count          2487
user_created_at      2487
verified             2487
protected            2487
original             2487
quote                2487
reply                2487
retweet              2487
tweets_in_dataset    2487
dtype: int64

In [10]:
user_summary_df.head()

,screen_name,name,organization,position,gender,followers_count,following_count,tweet_count,user_created_at,verified,protected,original,quote,reply,retweet,tweets_in_dataset
user_id,,,,,,,,,,,,,,,,
23455653,abettel,"Bettelheim, Adriel",Politico,Health Care Editor,F,2664,1055,15990,Mon Mar 09 16:32:20 +0000 2009,True,False,289.00,12.00,6.00,52.00,359.00
33919343,AshleyRParker,"Parker, Ashley",Washington Post,White House Reporter,F,122382,2342,12433,Tue Apr 21 14:28:57 +0000 2009,True,False,172.00,67.00,11.00,120.00,370.00
18580432,b_fung,"Fung, Brian",Washington Post,Tech Reporter,M,16558,2062,44799,Sat Jan 03 15:15:57 +0000 2009,True,False,257.00,85.00,205.00,82.00,629.00
399225358,b_muzz,"Murray, Brendan",Bloomberg News,"Managing Editor, U.S. Economy",M,624,382,360,Thu Oct 27 05:34:05 +0000 2011,True,False,3.00,0.00,0.00,5.00,8.00
18834692,becca_milfeld,"Milfeld, Becca",Agence France-Presse,English Desk Editor and Journalist,F,483,993,1484,Sat Jan 10 13:58:43 +0000 2009,False,False,3.00,14.00,0.00,7.00,24.00


### Remove users with no tweets in dataset

In [11]:
user_summary_df[user_summary_df.tweets_in_dataset == 0].count()

screen_name          195
name                 195
organization         195
position             195
gender               194
followers_count      195
following_count      195
tweet_count          195
user_created_at      195
verified             195
protected            195
original             195
quote                195
reply                195
retweet              195
tweets_in_dataset    195
dtype: int64

In [12]:
user_summary_df = user_summary_df[user_summary_df.tweets_in_dataset != 0]
user_summary_df.count()

screen_name          2292
name                 2292
organization         2292
position             2289
gender               2292
followers_count      2292
following_count      2292
tweet_count          2292
user_created_at      2292
verified             2292
protected            2292
original             2292
quote                2292
reply                2292
retweet              2292
tweets_in_dataset    2292
dtype: int64

## Tweeter analysis

### Gender

In [13]:
pd.DataFrame({'count':user_summary_df.gender.value_counts(), 'percentage':user_summary_df.gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

,count,percentage
M,1299,56.7%
F,993,43.3%


### Summary

#### All

In [14]:
user_summary_df[['followers_count', 'following_count', 'tweet_count', 'original', 'quote', 'reply', 'retweet', 'tweets_in_dataset']].describe()

,followers_count,following_count,tweet_count,original,quote,reply,retweet,tweets_in_dataset
count,"2,292.00","2,292.00","2,292.00","2,292.00","2,292.00","2,292.00","2,292.00","2,292.00"
mean,"16,467.62","1,444.83","9,619.69",102.06,48.73,55.08,150.64,356.52
std,"91,886.90","3,003.00","16,618.09",169.43,135.90,249.18,585.08,833.76
min,6.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
25%,831.75,505.75,"1,449.50",10.00,1.00,1.00,8.00,32.00
50%,"2,419.50",998.50,"4,211.50",41.00,9.00,5.00,39.00,122.00
75%,"7,348.75","1,713.50","10,817.25",124.25,43.00,30.00,129.00,375.00
max,"2,176,578.00","96,194.00","208,763.00","2,693.00","3,069.00","9,033.00","21,524.00","21,547.00"


#### Female

In [15]:
user_summary_df[user_summary_df.gender == 'F'][['followers_count', 'following_count', 'tweet_count', 'original', 'quote', 'reply', 'retweet', 'tweets_in_dataset']].describe()

,followers_count,following_count,tweet_count,original,quote,reply,retweet,tweets_in_dataset
count,993.00,993.00,993.00,993.00,993.00,993.00,993.00,993.00
mean,"11,609.53","1,314.07","7,498.74",83.84,39.27,32.06,135.55,290.72
std,"65,563.72","1,250.56","11,312.72",124.86,135.05,94.73,724.92,833.07
min,6.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00
25%,825.00,567.00,"1,393.00",8.00,1.00,1.00,9.00,32.00
50%,"2,327.00","1,034.00","4,055.00",39.00,9.00,4.00,37.00,111.00
75%,"6,340.00","1,659.00","8,983.00",111.00,33.00,21.00,115.00,314.00
max,"1,388,543.00","18,197.00","118,713.00","1,440.00","3,069.00","1,458.00","21,524.00","21,547.00"


#### Male

In [16]:
user_summary_df[user_summary_df.gender == 'M'][['followers_count', 'following_count', 'tweet_count', 'original', 'quote', 'reply', 'retweet', 'tweets_in_dataset']].describe()

,followers_count,following_count,tweet_count,original,quote,reply,retweet,tweets_in_dataset
count,"1,299.00","1,299.00","1,299.00","1,299.00","1,299.00","1,299.00","1,299.00","1,299.00"
mean,"20,181.31","1,544.78","11,241.02",115.99,55.96,72.69,162.17,406.81
std,"107,635.37","3,833.89","19,584.46",195.72,136.16,319.41,449.75,831.10
min,10.00,0.00,5.00,0.00,0.00,0.00,0.00,1.00
25%,857.50,472.00,"1,477.00",12.00,0.00,1.00,6.00,33.00
50%,"2,498.00",953.00,"4,401.00",44.00,9.00,6.00,40.00,131.00
75%,"8,341.50","1,763.00","12,584.50",140.00,50.50,38.50,142.00,428.00
max,"2,176,578.00","96,194.00","208,763.00","2,693.00","1,955.00","9,033.00","7,528.00","11,432.00"


### Verified

#### All

In [17]:
pd.DataFrame({'count':user_summary_df.verified.value_counts(), 'percentage':user_summary_df.verified.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

,count,percentage
True,1240,54.1%
False,1052,45.9%


#### Female

In [18]:
pd.DataFrame({'count':user_summary_df[user_summary_df.gender == 'F'].verified.value_counts(), 'percentage':user_summary_df[user_summary_df.gender == 'F'].verified.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

,count,percentage
True,512,51.6%
False,481,48.4%


#### Male

In [19]:
pd.DataFrame({'count':user_summary_df[user_summary_df.gender == 'M'].verified.value_counts(), 'percentage':user_summary_df[user_summary_df.gender == 'M'].verified.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

,count,percentage
True,728,56.0%
False,571,44.0%


## Mention data prep

### Load mentions from tweets
Including original tweets only

In [20]:
%matplotlib inline
import pandas as pd
import numpy as np
import logging
from dateutil.parser import parse as date_parse
from utils import load_tweet_df, tweet_type
import matplotlib.pyplot as plt


logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Set float format so doesn't display scientific notation
pd.options.display.float_format = '{:20,.2f}'.format

# Simply the tweet on load
def mention_transform(tweet):
    mentions = []
    if tweet_type(tweet) == 'original':
        for mention in tweet.get('entities', {}).get('user_mentions', []):
            mentions.append({
                'tweet_id': tweet['id_str'],
                'user_id': tweet['user']['id_str'],
                'screen_name': tweet['user']['screen_name'],
                'mention_user_id': mention['id_str'],
                'mention_screen_name': mention['screen_name'],
                'tweet_created_at': date_parse(tweet['created_at'])
            })
    return mentions

mention_df = load_tweet_df(mention_transform, ['tweet_id', 'user_id', 'screen_name', 'mention_user_id',
                                           'mention_screen_name', 'tweet_created_at'], 
                           dedupe_columns=['tweet_id', 'mention_user_id'])
mention_df.count()

INFO:root:Loading from tweets/41feff28312c433ab004cd822212f4c2_001.json.gz
DEBUG:root:Loaded 50000
DEBUG:root:Loaded 100000
DEBUG:root:Loaded 150000
DEBUG:root:Loaded 200000
DEBUG:root:Loaded 250000
DEBUG:root:Loaded 300000
INFO:root:Loading from tweets/642bf140607547cb9d4c6b1fc49772aa_001.json.gz
DEBUG:root:Loaded 350000
DEBUG:root:Loaded 400000
DEBUG:root:Loaded 450000
DEBUG:root:Loaded 500000
DEBUG:root:Loaded 550000
INFO:root:Loading from tweets/9f7ed17c16a1494c8690b4053609539d_001.json.gz
DEBUG:root:Loaded 600000
DEBUG:root:Loaded 650000
DEBUG:root:Loaded 700000
DEBUG:root:Loaded 750000
DEBUG:root:Loaded 800000


tweet_id               118210
user_id                118210
screen_name            118210
mention_user_id        118210
mention_screen_name    118210
tweet_created_at       118210
dtype: int64

In [21]:
mention_df.head()

,tweet_id,user_id,screen_name,mention_user_id,mention_screen_name,tweet_created_at
0,874695069584027648,15731368,HowardKurtz,110445334,megynkelly,2017-06-13 18:28:50+00:00
1,874695069584027648,15731368,HowardKurtz,171154131,HappeningNow,2017-06-13 18:28:50+00:00
2,874652287809064960,15731368,HowardKurtz,110445334,megynkelly,2017-06-13 15:38:50+00:00
3,874281339150884864,15731368,HowardKurtz,486907980,CarleyShimkus,2017-06-12 15:04:49+00:00
4,874265555552882688,15731368,HowardKurtz,2263438704,GillianHTurner,2017-06-12 14:02:06+00:00


### Distinct tweets

In [22]:
mention_df['tweet_id'].unique().size

84942

## Mentioned analysis
That is, who is mentioned not who is mentioning.

*Note that for each of these, the complete list is being written to CSV in the output directory.*

### All
This is based on screen name, which could have changed during collection period. However, for the users that would be at the top of this list, seems unlikely.

In [23]:
all_mentioned_df = mention_df.mention_screen_name.value_counts()
all_mentioned_df.to_csv('output/all_mentioned.csv')
all_mentioned_df.head(25)

realDonaldTrump    2876
POTUS              2265
wusa9              2111
AP                 1948
USATODAY           1235
nbcwashington      1230
WSJ                1227
dcexaminer         1034
SHSanders45         927
nytimes             829
BloombergBNA        759
politico            747
SpeakerRyan         700
Scaramucci          657
PressSec            654
CNN                 628
ABC7News            604
SenJohnMcCain       599
WTOP                529
BloombergLaw        517
VP                  506
SteveScalise        505
MSNBC               486
Reuters             483
bpolitics           432
Name: mention_screen_name, dtype: int64

### Beltway journalists

In [24]:
journalists_mention_df = mention_df.join(user_summary_df['gender'], how='inner', on='mention_user_id')
journalists_mention_df.count()

tweet_id               14298
user_id                14298
screen_name            14298
mention_user_id        14298
mention_screen_name    14298
tweet_created_at       14298
gender                 14298
dtype: int64

In [25]:
journalists_mention_df.head()

,tweet_id,user_id,screen_name,mention_user_id,mention_screen_name,tweet_created_at,gender
9,873955901484486656,15731368,HowardKurtz,16157855,edhenry,2017-06-11 17:31:38+00:00,M
14,873877145117962241,15731368,HowardKurtz,16157855,edhenry,2017-06-11 12:18:41+00:00,M
6414,874439120344535041,19455864,finnygo,16157855,edhenry,2017-06-13 01:31:47+00:00,M
23175,879021641141022720,15731368,HowardKurtz,16157855,edhenry,2017-06-25 17:01:05+00:00,M
23178,878956408657633280,15731368,HowardKurtz,16157855,edhenry,2017-06-25 12:41:52+00:00,M


#### All beltway journalists

In [26]:
all_journalists_mentioned_df = journalists_mention_df.mention_screen_name.value_counts()
all_journalists_mentioned_df.to_csv('output/all_journalists_mentioned.csv')
all_journalists_mentioned_df.head(25)

AllysonRaeWx       330
TenaciousTopper    239
hbwx               235
burgessev          212
jenhab             200
seungminkim        143
jaketapper         127
WaPoSean           117
pkcapitol          116
DanaBashCNN        115
kelsey_snell       109
peterbakernyt      107
daveweigel         106
StevenTDennis      105
jonkarl            104
AshleyRParker      100
reporterjoe         98
mkraju              95
ZoeTillman          87
edatpost            84
HopeSeck            83
HardballChris       80
GlennThrush         78
jameshohmann        78
BresPolitico        78
Name: mention_screen_name, dtype: int64

In [27]:
pd.DataFrame({'count':journalists_mention_df.gender.value_counts(), 'percentage':journalists_mention_df.gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

,count,percentage
M,8298,58.0%
F,6000,42.0%


#### Female beltway journalists

In [28]:
female_journalists_mentioned_df = journalists_mention_df[journalists_mention_df.gender == 'F'].mention_screen_name.value_counts()
female_journalists_mentioned_df.to_csv('output/female_journalists_mentioned.csv')
female_journalists_mentioned_df.head(25)

AllysonRaeWx     330
jenhab           200
seungminkim      143
DanaBashCNN      115
kelsey_snell     109
AshleyRParker    100
ZoeTillman        87
HopeSeck          83
jestei            76
morningmika       70
kasie             67
eilperin          67
mj_lee            67
pw_cunningham     67
FoxReports        65
LauraLitvan       58
margarettalev     58
sarahkliff        57
caitlinnowens     57
rachaelmbade      56
juliehdavis       55
rachanadixit      55
Oriana0214        55
jpaceDC           52
JudyWoodruff      49
Name: mention_screen_name, dtype: int64

#### Male beltway journalists

In [29]:
male_journalists_mentioned_df = journalists_mention_df[journalists_mention_df.gender == 'M'].mention_screen_name.value_counts()
male_journalists_mentioned_df.to_csv('output/male_journalists_mentioned.csv')
male_journalists_mentioned_df.head(25)

TenaciousTopper    239
hbwx               235
burgessev          212
jaketapper         127
WaPoSean           117
pkcapitol          116
peterbakernyt      107
daveweigel         106
StevenTDennis      105
jonkarl            104
reporterjoe         98
mkraju              95
edatpost            84
HardballChris       80
jameshohmann        78
BresPolitico        78
GlennThrush         78
jmartNYT            75
chrisgeidner        73
kenvogel            67
BretBaier           66
Acosta              61
pauldemko           60
danbalz             57
mikedebonis         56
Name: mention_screen_name, dtype: int64